# Goals
**[Script]** This script does the following:
1. Plot gene and transcript counts across references, by chromosome, and by biotype
2. Plot FASTA stats, such as number of ambiguous bases
3. Plot overlapping features, such as common and distinct genes across references
4. Write out Excel files with counts, plot links, and common and distinct features

**[Overall]** Feature differences between references, such as different gene sequneces, can influence downsteam results for alignment, variant calling, and more

**[To Do]**
1. Plot overlaps with `ComplexUpset` to add bar chart (ex. validated vs. unvalidated transcripts)

# Packages

In [2]:
#####################
### Data Cleaning ###
#####################
library("data.table")
library("openxlsx")
library("janitor")
library("rlang")

###########
### Viz ###
###########
library("tidyverse")
library("ggrepel")
library("UpSetR")
library("ComplexUpset")

####################
### Session Info ###
####################
library("sessioninfo")

## Options

In [ ]:
options(warn = 1)
options(repr.matrix.max.cols=200, repr.matrix.max.rows=200)

theme_set(theme_bw(base_size=18, base_family="sans"))

## Functions

In [ ]:
source("./References.R")

# Parameters

## Inputs

In [ ]:
species = "Homo_sapiens"  # Genus_species or species1_and_species2
primary_chr = as.character(c(1:22,"X","Y"))
output_path = file.path("../outputs", species)

ref_names = c("Ensembl GRCh38","RefSeq GRCh38","RefSeq T2T")  # Order in all plots
ref_colors = c("#008080","#DAA520","#6A5ACD")  # teal, goldenrod, slateblue (purplish)

## Outputs

In [ ]:
n_refs = length(ref_names)

plot_path = file.path("../results/plots",species)
dir.create(plot_path, recursive=TRUE, showWarnings=FALSE)

excel_file = paste0("../results/Reference_counts_", species, ".xlsx")

# Plot genes
- **Note:** Since RefSeq GTF files (and ann/bed files) only have gene names, use names when comparing to Ensembl
    - Some Ensembl genes are in more than one place (ex. DDX11L2 is on chrs 1 and 2, but ENSG IDs are unique)
- **Note:** Since we are plotting genes **by gene name**, the overall gene counts are an underestimate (ex. no pseudogenes)
- **QC:** Gene and transcript counts don't exactly match with [Ensembl](https://useast.ensembl.org/Homo_sapiens/Info/Annotation) and [RefSeq](https://www.ncbi.nlm.nih.gov/refseq/annotation_euk/Homo_sapiens/GCF_009914755.1-RS_2024_08/) but they are close (+/- 200 features)

In [ ]:
# Use gene annotation files to plot gene counts and gene types
gene_ann_paths = get_paths(ref_names, file_id="gene.sorted.tsv", ref_path=output_path)

df_gene = concat_files(gene_ann_paths, ref_names, ref_colors, to_chr=primary_chr)

dim(df_gene)
head(df_gene, n=3)

## Genes per reference

In [ ]:
df_unique_genes = get_unique_feats(df_gene, "gene_name")
df_total_genes = get_totals(df_unique_genes)

fig_gene = plot_ref_counts(
    df_total_genes,
    title_text = paste("Total gene counts in",n_refs,"reference genomes"),
    subtitle_text = "RefSeq GRCh38 has the most genes (by name)",
    out_file = file.path(plot_path, "gene_counts_per_reference.png"),
    w=6, h=3
)

IRdisplay::display_html(paste0("<img src=",fig_gene$path, ">"))

## Genes per chromosome

In [ ]:
df_chr_genes = get_totals_per_chr(df_unique_genes)

fig_gene_chr = plot_ref_counts(
    df_chr_genes,
    title_text = paste("Gene counts per chromosome in",n_refs,"reference genomes"),
    subtitle_text = "Chr1 has the most genes; RefSeq T2T has the best resolution on chrY",
    out_file = file.path(plot_path, "gene_counts_per_chr.png"),
    w=9, h=9
)

IRdisplay::display_html(paste0("<img src=",fig_gene_chr$path, ">"))

## Gene types

In [ ]:
df_total_gene_types = get_totals_per_type(df_unique_genes, "gene_name", types_to_plot=c("protein_coding"))

fig_gene_type = plot_ref_counts(
    df_total_gene_types,
    title_text = paste("Protein-coding genes in",n_refs,"reference genomes"),
    subtitle_text = "QC: Counts almost match those on websites",
    out_file = file.path(plot_path, "gene_types_per_reference.png"),
    w=6, h=3
)

IRdisplay::display_html(paste0("<img src=",fig_gene_type$path, ">"))

# Plot transcripts
- **Note:** Since we are plotting transcripts *by transcript ID*, the overall counts should be accurate

In [ ]:
# Use transcript annotation files to plot transcript counts and types
tx_ann_paths = get_paths(ref_names, file_id="transcript.sorted.tsv", ref_path=output_path)

df_tx = concat_files(tx_ann_paths, ref_names, ref_colors, to_chr=primary_chr)

dim(df_tx)
head(df_tx, n=3)

## Transcripts per reference

In [ ]:
df_unique_tx = get_unique_feats(df_tx, "transcript_id") %>%
    mutate(transcript_biotype = recode(
        transcript_biotype, "mRNA" = "protein_coding", "lnc_RNA" = "lncRNA"))

df_total_tx = get_totals(df_unique_tx)

fig_tx = plot_ref_counts(
    df_total_tx,
    title_text = paste("Total transcripts in",n_refs,"reference genomes"),
    subtitle_text = "Ensembl GRCh38 has the most transcripts",
    out_file = file.path(plot_path, "transcript_counts_per_reference.png"),
    w=6, h=3
)

IRdisplay::display_html(paste0("<img src=",fig_tx$path, ">"))

## Transcripts per chromosome

In [ ]:
df_chr_tx = get_totals_per_chr(df_unique_tx)

fig_tx_chr = plot_ref_counts(
    df_chr_tx,
    title_text = paste("Transcripts per chromosome in",n_refs,"reference genomes"),
    subtitle_text = "Chr1 has the most transcripts; RefSeq T2T has the best resolution on chrY",
    out_file = file.path(plot_path, "transcript_counts_per_chr.png"),
    w=9, h=9
)

IRdisplay::display_html(paste0("<img src=",fig_tx_chr$path, ">"))

## Transcript types

In [ ]:
df_total_tx_types = get_totals_per_type(df_unique_tx, "transcript_id", types_to_plot=c("protein_coding"))

fig_tx_type = plot_ref_counts(
    df_total_tx_types,
    title_text = paste("Protein-coding transcripts in",n_refs,"reference genomes"),
    subtitle_text = "Ensembl has less protein-coding due to more lncRNA (not shown)",
    out_file = file.path(plot_path, "transcript_types_per_reference.png"),
    w=6, h=3
)

IRdisplay::display_html(paste0("<img src=",fig_tx_type$path, ">"))

# Plot FASTA stats
- Plot number of ambiguous bases *found in primary assemblies*, since this is a proxy for overall FASTA quality
    - `sum_n`: Column containing number of ambiguous letters found in a FASTA file (ex. N, X, K)
    - Note: Additional FASTA files and stats can be plotted by updating filter logic (sum_len) and count column (sum_n)

In [ ]:
stats_paths = get_paths(ref_names, file_id="fasta_stats.tsv", ref_path=output_path)

df_amb_bases = concat_fasta_stats(stats_paths, ref_names, ref_colors)

fig_stats = plot_ref_counts(
    df_amb_bases,
    title_text = paste("Total ambiguous bases (N,X) in",n_refs,"reference genomes"),
    subtitle_text = "RefSeq T2T has by far the best annotation",
    out_file = file.path(plot_path, "ambigous_bases_per_reference.png"),
    w=6, h=3
)

IRdisplay::display_html(paste0("<img src=",fig_stats$path, ">"))

# Plot overlapping features

In [ ]:
df_gene_binary = get_intersections(df_unique_genes)
fig_gene_overlap = plot_common_features(df_gene_binary, "gene_name")
IRdisplay::display_html(paste0("<img src=",fig_gene_overlap$path, ">"))

In [ ]:
# Find overlapping transcripts for RefSeq references, since Ensembl uses a different ID
df_refseq_unique_tx = df_unique_tx %>% filter(ref_name %in% c("RefSeq GRCh38","RefSeq T2T"))

df_tx_binary = get_intersections(df_refseq_unique_tx)
fig_tx_overlap = plot_common_features(df_tx_binary, "transcript_id")
IRdisplay::display_html(paste0("<img src=",fig_tx_overlap$path, ">"))

# Write out

In [ ]:
# Sheet for total counts
df_counts = list(df_total_genes %>% select(ref_name, Count) %>% rename(total_genes = Count),
                 df_total_gene_types %>% select(ref_name, Count) %>% rename(total_protein_coding_genes = Count),
                 df_total_tx %>% select(ref_name, Count) %>% rename(total_transcripts = Count),
                 df_total_tx_types %>% select(ref_name, Count) %>% rename(total_protein_coding_transcripts = Count),
                 df_amb_bases %>% select(ref_name, Count) %>% rename(total_ambiguous_bases = Count)) %>%
    reduce(full_join, by=c("ref_name"))

plot_links = gsub("../results/","", c(fig_gene$path, fig_gene_type$path, fig_tx$path, fig_tx_type$path, fig_stats$path))
plot_links = c(c("Plot Links"), paste0('=HYPERLINK("',plot_links,'","Link")'))

wb_1 = create_excel_tab(
    df_counts,
    tab_name = "All Primaries",
    out_file = NULL,
    prev_workbook = NULL,
    template_file = NULL,
    plot_col = NULL,
    plot_row = plot_links,
    freeze_cell = c(1,2),
    header_color = "#F7FFBF"
)

In [ ]:
# Sheet for chromosome counts
df_counts_chr = list(df_chr_genes %>% select(ref_name, chr, Count) %>% rename(total_genes = Count),
                     df_chr_tx %>% select(ref_name, chr, Count) %>% rename(total_transcripts = Count)) %>%
    reduce(full_join, by=c("ref_name", "chr")) %>%
    arrange(match(chr, primary_chr)) %>%
    mutate(ref_name=as.character(ref_name), chr=as.character(chr))

plot_links_chr = gsub("../results/","", c(fig_gene_chr$path ,fig_tx_chr$path))
plot_links_chr = c(c("Plot Links", "By Chromosome"), paste0('=HYPERLINK("',plot_links_chr,'","Link")'))

wb_2 = create_excel_tab(
    df_counts_chr,
    tab_name = "By Chromosome",
    out_file = NULL,
    prev_workbook = wb_1,
    template_file = NULL,
    plot_col = NULL,
    plot_row = plot_links_chr,
    freeze_cell = c(1,2),
    header_color = "#F7FFBF"
)

In [ ]:
# Sheet for overlapping genes and transcripts
overlap_gene_lists = upset_data(df_gene_binary, intersect=colnames(df_gene_binary))
intersect_gene_list = overlap_gene_lists$sizes$exclusive_intersection
df_gene_overlap = tibble(intersection=names(intersect_gene_list), common_gene_names=intersect_gene_list)

overlap_tx_lists = upset_data(df_tx_binary, intersect=colnames(df_tx_binary))
intersect_tx_list = overlap_tx_lists$sizes$exclusive_intersection
df_tx_overlap = tibble(intersection=names(intersect_tx_list), common_transcript_ids=intersect_tx_list)

df_overlap = full_join(df_gene_overlap, df_tx_overlap, by="intersection") %>% replace(is.na(.), 0)

plot_links_overlap = gsub("../results/","", c(fig_gene_overlap$path, fig_tx_overlap$path))
plot_links_overlap = c(c("Plot Links"), paste0('=HYPERLINK("',plot_links_overlap,'","Link")'))

wb_3 = create_excel_tab(
    df_overlap,
    tab_name = "By Features",
    out_file = NULL,
    prev_workbook = wb_2,
    template_file = NULL,
    plot_col = NULL,
    plot_row = plot_links_overlap,
    freeze_cell = c(1,2),
    header_color = "#F7FFBF"
)

In [ ]:
df_common_genes = summarize_common_features(df_unique_genes) %>% rename(gene_name = feat_id)

wb_4 = create_excel_tab(
    df_common_genes,
    tab_name = "Common Genes",
    out_file = NULL,
    prev_workbook = wb_3,
    template_file = NULL,
    plot_col = NULL,
    plot_row = NULL,
    freeze_cell = c(1,2),
    header_color = "#F7FFBF"
)

In [ ]:
df_common_tx = summarize_common_features(df_refseq_unique_tx) %>% rename(transcript_id = feat_id)

wb_5 = create_excel_tab(
    df_common_tx,
    tab_name = "Common Transcripts",
    out_file = excel_file,
    prev_workbook = wb_4,
    template_file = NULL,
    plot_col = NULL,
    plot_row = NULL,
    freeze_cell = c(1,2),
    header_color = "#F7FFBF"
)

# Session info

In [3]:
si = session_info(pkg="attached", to_file=FALSE)
as_tibble(si$platform) %>% select(version, os, system, date)
as_tibble(si$packages) %>% select(package, loadedversion)

version,os,system,date
<chr>,<chr>,<chr>,<chr>
R version 4.4.0 (2024-04-24),Ubuntu 22.04.4 LTS,"x86_64, linux-gnu",2025-08-11


package,loadedversion
<chr>,<chr>
ComplexUpset,1.3.3
data.table,1.15.0
dplyr,1.1.4
forcats,1.0.0
ggplot2,3.5.1
ggrepel,0.9.6
janitor,2.2.0
lubridate,1.9.3
openxlsx,4.2.7.1
